In [1]:
!pip install -q accelerate
!pip install git+https://github.com/huggingface/transformers

^C
^C


In [2]:
!pip install ctransformers

^C


In [7]:
!nvidia-smi

Mon Mar  4 12:24:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0             13W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
import accelerate 
import torch
import transformers
import json

CHECKPOINT = "MaziyarPanahi/gemma-7b-GGUF"

# Make sure you set this variable according to the environment
environment = "local"

AttributeError: module 'torch' has no attribute 'version'

In [ ]:
# Show that when we do this neither GPU nor CPU memory increases
config = transformers.AutoConfig.from_pretrained(CHECKPOINT)

with accelerate.init_empty_weights():
    fake_model = transformers.AutoModelForCausalLM.from_config(config)
    
device_map = accelerate.infer_auto_device_map(fake_model, max_memory={0: "3GiB", "cpu": "6GiB"})
print(json.dumps(device_map, indent=4))

In [ ]:
device_map = {
    "model.embed_tokens": 0,
    "model.layers.0": 0,
    "model.layers.1": 0,
    "model.layers.2.self_attn.q_proj": 0,
    "model.layers.2.self_attn.k_proj": 0,
    "model.layers.2.self_attn.v_proj": 0,
    "model.layers.2.self_attn.o_proj": "cpu",
    "model.layers.2.self_attn.rotary_emb": "cpu",
    "model.layers.2.mlp": "cpu",
    "model.layers.2.input_layernorm": "cpu",
    "model.layers.2.post_attention_layernorm": "cpu",
    "model.layers.3": "cpu",
    "model.layers.4": "cpu",
    "model.layers.5": "cpu",
    "model.layers.6": "cpu",
    "model.layers.7.self_attn": "cpu",
    "model.layers.7.mlp.gate_proj": "cpu",
    "model.layers.7.mlp.up_proj": "cpu",
    "model.layers.7.mlp.down_proj": "disk",
    "model.layers.7.mlp.act_fn": "disk",
    "model.layers.7.input_layernorm": "disk",
    "model.layers.7.post_attention_layernorm": "disk",
    "model.layers.8": "disk",
    "model.layers.9": "disk",
    "model.layers.10": "disk",
    "model.layers.11": "disk",
    "model.layers.12": "disk",
    "model.layers.13": "disk",
    "model.layers.14": "disk",
    "model.layers.15": "disk",
    "model.layers.16": "disk",
    "model.layers.17": "disk",
    "model.layers.18": "disk",
    "model.layers.19": "disk",
    "model.layers.20": "disk",
    "model.layers.21": "disk",
    "model.layers.22": "disk",
    "model.layers.23": "disk",
    "model.layers.24": "disk",
    "model.layers.25": "disk",
    "model.layers.26": "disk",
    "model.layers.27": "disk",
    "model.layers.28": "disk",
    "model.layers.29": "disk",
    "model.layers.30": "disk",
    "model.layers.31": "disk",
    "model.norm": "disk",
    "lm_head": "disk"
}

In [ ]:
import transformers
from ctransformers import AutoModelForCausalLM, AutoTokenizer

# Check what happens when device_map = auto
# This will fail as the model in FP32 precision cannot be fit on CPU
# model = transformers.LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")
# This will fail as the model in FP32 precision cannot be fit on GPU
# model = transformers.LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf").to("cuda")
# This will also fail as the model cannot be fit on GPU fully even with the quantization
# model = transformers.LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", device_map="auto", load_in_8bit=True)

if environment == "local":
    model = AutoModelForCausalLM.from_pretrained(
        CHECKPOINT, 
        # load_in_8bit=True,
        # load_in_4bit=True,
        # llm_int8_enable_fp32_cpu_offload=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model)